In [ ]:
import os
import numpy as np
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from skimage.feature import hog
from skimage import exposure
from tensorflow.keras.models import Model
def extract_features_from_directory(directory, deep_target_size=(299, 299), hog_pixels_per_cell=(8, 8)):
    deep_features = []
    hog_features = []
    labels = [] 
    class_labels = sorted(os.listdir(directory))  
    model = InceptionResNetV2(weights='imagenet')
    base_model = Model(inputs=model.input, outputs=model.get_layer('predictions').output)
    for i, class_label in enumerate(class_labels):
        class_path = os.path.join(directory, class_label)       
        for filename in os.listdir(class_path):
            img_path = os.path.join(class_path, filename)
            img_deep = image.load_img(img_path, target_size=deep_target_size)
            img_array_deep = image.img_to_array(img_deep)
            img_array_deep = np.expand_dims(img_array_deep, axis=0)
            img_array_deep = preprocess_input(img_array_deep)
            img_hog = image.load_img(img_path)
            img_array_hog = image.img_to_array(img_hog)
            img_array_hog = resize(img_array_hog, (img_array_hog.shape[0] // hog_pixels_per_cell[0] * hog_pixels_per_cell[0],
                                                   img_array_hog.shape[1] // hog_pixels_per_cell[1] * hog_pixels_per_cell[1]))
            deep_features.append(base_model.predict(img_array_deep).flatten())
            gray_img = np.dot(img_array_hog, [0.2989, 0.5870, 0.1140])
            hog_feature_vector, _ = hog(gray_img, orientations=9, pixels_per_cell=hog_pixels_per_cell, 
                                        cells_per_block=(2, 2), visualize=True)
            hog_features.append(hog_feature_vector)       
            labels.append(i)  
    return np.array(deep_features), np.array(hog_features), np.array(labels)
dataset_directory = 'path_to_dataset'  # Update this with your dataset path
deep_features, hog_features, labels = extract_features_from_directory(dataset_directory)
pca1 = PCA(n_components=500)  
deep_features_pca1 = pca1.fit_transform(deep_features)
sorted_indices_deep = np.argsort(pca1.explained_variance_, axis=0)[::-1]
pca2 = PCA(n_components=500)  
hog_features_pca2 = pca2.fit_transform(hog_features)
sorted_indices_hog = np.argsort(pca2.explained_variance_, axis=0)[::-1]

In [ ]:
deep_features_pca_sorted=deep_features_pca1[:, sorted_indices_deep]
hog_features_pca_sorted=hog_features_pca2[:, sorted_indices_hog]
top_components = 200
deep_features_top = deep_features_pca_sorted[:, :top_components]
hog_features_top = hog_features_pca_sorted[:, :top_components]
all_features = np.concatenate((deep_features_top, hog_features_top), axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_features,labels,test_size=0.2, random_state=42, stratify=labels)
svm_model = SVC(kernel='rbf', C=2, gamma='scale')
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)
print("\nAccuracy:", accuracy)
print("\nClassification Report:\n", classification_rep)